## Pricing of american option under the CRR model - Constantin Gleyze

The goal of this project is to propose an algorithm to price options within the Cox-Ross-Rubinstein (CRR) paradigm. 

A sepcific example for americcan put will be done. 

We therefore consider a put of maturity $T$. The holder can decide to exercice it at every step of discretisation. 

More precisely, for $T$, we discretize with $N$ step equally. 

For each $n$  in {$0$,....,$N$}, the holder can exercise at $t_n:=n\Delta t=n\times \frac{T}{N}$. 

We remember the backward algorithm used in the CRR model to price american options. 

We fix: 
- $V_n(.)$ the value of the option
- $X_n$ the value of our underlying at each time $n$  in {$0$,....,$N$}
- $Z_n =\phi_n(X)$ the actualized payoff of our option at time $t_n$

We are looking for $V_0(X_0) = \sup_{\tau \in \mathcal{T}_0^N}\mathbb{E}_{\mathbb{Q}}[Z_\tau]$ with $\tau \in \mathcal{T}_0^N$ the set of stopping times with $\mathbb{Q}$  always surely values in {$t_0$,....,$t_N$}. 

In the CRR model, the underlying can increase from a factor $u$ or decrease from a factor $d$ = $1/u$. 

The underlying process $(X_n)_{n=0,\dots,N}$ is a Markov Chain with transition kernel $(P_n)_{n=0,\dots,N-1}$ such that, by denoting $E$ the set of all possible value of $X$ : 

$\begin{equation*}
\forall 1 \leq n\leq {N}, \forall x, y \in E, \quad
P_{n+1}(x, y) = \mathbb{Q}(X_{n+1} = y|X_{n}=x) = \begin{cases}
q & \text{if } \frac{y}{x} = u, \\
1-q & \text{if } \frac{y}{x} = d, \\
0 & \text{otherwise} \\
\end{cases}
\end{equation*}$

Within the non-arbitrage hypothesis, we have a unique neutral risk probability $\mathbb{Q}$ such that : 

$q = \frac{\exp(r\Delta t)-d}{u-d}$ with $r$ the risk-free rate. 

To compute the value of the option, we use the following backward algorithm : 

$\begin{equation*}
\begin{cases}
V_N(x)&=& \phi_N(x) \\
V_n(x)& =& \max(\phi_n(x),P_{n+1}V_{n+1}(x)), \quad 0\leq n \leq N-1
\end{cases}
\end{equation*}$

With $P_{n+1}V_{n+1}(x) = \mathbb{E}[V_{n+1}(X_{n+1})\mid X_n = x]$. 

In the case of E finite (our case), it can be rewritten as : $P_{n+1}V_{n+1}(x) = \sum_{y \in E} P_{n+1}(x, y) V_{n+1}(y)$. 

We implement the above algorithm. 

In [90]:
#Import of necessary librairies 
import numpy as np
import scipy.sparse

In [91]:
#We define our methods through a class

class CRR(): 

    #Initialisation of parameters
    def __init__ (self, N,T,r,K,div, u,d,S0): 
        self.__N = N
        self.__T = T
        self.__r = r
        self.__K = K
        self.__div = div
        self.__dt = self.__T/self.__N
        self.__u = u
        self.__d = d
        self.__q = (np.exp(self.__r*self.__dt)-self.__d) / (self.__u - self.__d)
        self.__S0 = S0
        self.__actu = np.exp(-(self.__r - self.__div)*self.__dt)

    #Generation of all our possibles prices 
    #Use of csr matrix to speed the next computations
    def price_matrix (self): 
        S = scipy.sparse.lil_matrix((self.__N+1, self.__N+1))  
        for i in range(self.__N+1):
            for j in range(i+1): 
                S[j, i] = self.__S0 * (self.__u ** (i-j)) * (self.__d ** (j))  
        return S.tocsr()  

    #Function to obtain the price by implementing the above backward algorithm
    def price_option(self,payoff_function): 
        S = self.price_matrix() #Generation of our prices 
        V = payoff_function(S[:,-1],self.__K) #We apply our payoff_function to our last prices
    
        for i in range(self.__N - 1, -1,-1): 
            
            size = i +1
            P = scipy.sparse.diags([self.__q, (1 - self.__q)],[0,1],shape=(size, size+1),format='csr') #We compute our transition kernel 
            markov_state = self.__actu*P@V # We compute our pondered set of possible values from time i-1 to i 
            intrinsec_value = payoff_function(S[:size,i],self.__K) #We compute the intrinsec value of the option i.e its value if exerced 
            V = markov_state.maximum(intrinsec_value) #We take the maximum and then return the price 
            
        return V[0,0]

By denoting $\sigma$ the volatility of our underlying $X$, we can observe a convergence with the Black Sholes prices by using : 

$u = e^{\sigma \sqrt{\Delta t}}$ with $\Delta t$ = $t_{i+1} - t_i$ for i in {$0$,....,$N-1$}. 

We will use a comparaison for the results the website https://www.coggit.com/freetools. 

In [92]:
def payoff_put(S_i,K):
    return scipy.sparse.csr_matrix(np.full(S_i.shape, K) - S_i).maximum(0)
def payoff_call(S_i,K):
    return scipy.sparse.csr_matrix(-np.full(S_i.shape, K) + S_i).maximum(0)

### ATM Put

We will price an At-The-Money (ATM) without dividends Put with the following parameters : 

- $r$ = 5%
- $\sigma$ = 20%
- $T$ = 1 year
- $S_0$ = 100
- $K$ = 100
- $N$ = 200

The exact price for the put is $6.10$. 

In [93]:
T = 1
sigma = 0.2
N = 200
 
dt = T/N
u = np.exp(sigma * np.sqrt(dt))  
d = 1 / u  

crr_model = CRR(N=N, T=T, r=0.05, K=100,div = 0, u=u, d=d, S0=100)


# Calcul du prix du Put
put_price = crr_model.price_option(payoff_function=payoff_put)
print(f" Put price ATM : {put_price:.4f}")

 Put price ATM : 6.0864


### OTM Put

We will price an Out-The-Money (OTM) Put without dividends with the following parameters : 

- $r$ = 5%
- $\sigma$ = 20%
- $T$ = 1 year
- $S_0$ = 100
- $K$ = 80
- $N$ = 200
 
The exact price for the put is $0.76$.

In [94]:
T = 1
sigma = 0.2
N = 200
 
dt = T/N
u = np.exp(sigma * np.sqrt(dt))  
d = 1 / u  

crr_model = CRR(N=N, T=T, r=0.05, K=80,div=0, u=u, d=d, S0=100)

# Calcul du prix du Put
put_price = crr_model.price_option(payoff_function=payoff_put)
print(f" Put price OTM : {put_price:.4f}")

 Put price OTM : 0.7228


### ITM Put

We will price an In-The-Money (ITM) Put without dividends with the following parameters : 

- $r$ = 5%
- $\sigma$ = 20%
- $T$ = 1 year
- $S_0$ = 100
- $K$ = 120
- $N$ = 200

The exact price for the put is $20.07$. 

In [95]:
T = 1
sigma = 0.2
N = 200
 
dt = T/N
u = np.exp(sigma * np.sqrt(dt))  
d = 1 / u  

crr_model = CRR(N=N, T=T, r=0.05, K=120, div=0,u=u, d=d, S0=100)

# Calcul du prix du Put
put_price = crr_model.price_option(payoff_function=payoff_put)
print(f" Put price ITM : {put_price:.4f}")

 Put price ITM : 20.1369


### With continuous dividends

This algorithm is no use for an american call option without dividends. Indeed without dividends, it is always optimal to exercice the american call option at maturity, matching the BS price of an european call. 

Because it is not always the optimal choice for american put option, we have a little more differences due to our algorithm. 

We will now introduce a continuous dividends $q$ : 

- $r$ = 5%
- $\sigma$ = 20%
- $T$ = 1 year
- $S_0$ = 100
- $K$ = 80,100,120
- $N$ = 200
- $q$ = 2%

In [103]:
T = 1
sigma = 0.2
N = 200
 
dt = T/N
u = np.exp(sigma * np.sqrt(dt))  
d = 1 / u  

crr_model_1 = CRR(N=N, T=T, r=0.05, K=100, div=0.02,u=u, d=d, S0=100)
crr_model_2 = CRR(N=N, T=T, r=0.05, K=120, div=0.02,u=u, d=d, S0=100)
crr_model_3 = CRR(N=N, T=T, r=0.05, K=80, div=0.02,u=u, d=d, S0=100)


# Calcul du prix du Put
call_price_1 = crr_model_1.price_option(payoff_function=payoff_call)
put_price_1 = crr_model_1.price_option(payoff_function=payoff_put)
call_price_2 = crr_model_2.price_option(payoff_function=payoff_call)
put_price_2 = crr_model_2.price_option(payoff_function=payoff_put)
call_price_3 = crr_model_3.price_option(payoff_function=payoff_call)
put_price_3 = crr_model_3.price_option(payoff_function=payoff_put)

For $K$ = 100 : 

The exact price for the call is $9.23$. 

The exact price for the put is $6.67$. 

In [104]:
print(f" Call price ATM : {call_price_1:.4f}")
print(f" Put price ATM : {put_price_1:.4f}")

 Call price ATM : 10.6515
 Put price ATM : 6.1544


For $K$ = 120 : 

The exact price for the call is $2.71$. 

The exact price for the put is $20.43$. 

In [105]:
print(f" Call price OTM : {call_price_2:.4f}")
print(f" Put price ITM : {put_price_2:.4f}")

 Call price OTM : 3.3172
 Put price ITM : 20.1819


For $K$ = 80 : 

The exact price for the call is $22.77$. 

The exact price for the put is $0.89$. 

In [106]:
print(f" Call price OTM : {call_price_3:.4f}")
print(f" Put price ITM : {put_price_3:.4f}")

 Call price OTM : 25.0839
 Put price ITM : 0.7344
